In [1]:
import pandas as pd
import os

In [2]:
# read in lineages
df_lin = pd.read_csv('bin/db/lineages.bed',sep='\t',index_col=1)
DB_LIN = df_lin.drop(columns=['Chromosome','End'])

In [3]:
OUTPUT = 'output/'
INPUT = 'input/'

In [ ]:
# rename files
directory = Path(INPUT)
files = [f for f in directory.iterdir() if f.is_file()]
for old_path in files:
    dir_name = os.path.dirname(old_path)
    base_name = os.path.basename(old_path)
    if os.path.splitext(base_name)[1] == '.csv':
        new_base = base_name.split('_')[0] + '.snp'
        new_path = os.path.join(dir_name, new_base)
        os.rename(old_path, new_path)
    else:
        pass

In [4]:
snp_files = os.listdir(INPUT+'snp/')

In [5]:
ref_dict = dict()
snp_dfs = dict()
snp_dfs_filtered = dict()

for file in snp_files:
    print(file)
    sample = file.replace('.snp','')
    df = pd.read_csv('input/snp/'+file)
    snp_dfs[sample] = df

TC00760619_S1.snp
TC00760619_S1.csv
TC00856554_S51.snp
TC00764189.snp
TC00831960_S32.snp
TC00855201_S148.snp
TC00862935_S189.snp
TC00836558_1.snp
TC00777418_S47.snp
TC00832902_S78.snp
TC00780707_S93.snp
TC00838386_S26.snp


In [6]:
# Variables

LIN_FREQ = 20
d_lin_name_number = dict(zip(DB_LIN['Lineage_number'],DB_LIN['Lineage_name']))

# create a lineage matrix that can later be used for viz if needed
lin_mat = pd.DataFrame()
for sample in snp_dfs:
    df = snp_dfs[sample]
    df = df.set_index('Reference Position')
    df = df.join(df_lin,rsuffix='_lin')
    df = df[(df['Allele'] == df['Allele_lin']) & (df['Reference'] == df['Ref'])]
    df = df[df['Frequency'] >= LIN_FREQ]
    df = df.groupby('Lineage_number').mean('Fequency')
    for i in df.index:
        lin_mat.at[i,sample] = df.at[i,'Frequency']
        
df = lin_mat.reset_index()
df = df.rename(columns={'index':'Lineage Number'})
df['Lineage Name'] = df['Lineage Number'].map(d_lin_name_number)
df = pd.melt(df,id_vars=['Lineage Name','Lineage Number'])
df = df[df['value'].notna()]
df = df.rename(columns={'variable':'Sample','value':'Frequency'})
df = df[['Sample','Lineage Name','Lineage Number','Frequency']]
#df.to_excel(OUTPUT+'/lineages.xlsx')